In [1]:
import tensorflow as tf
import matplotlib.pyplot as pylt
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import time

In [2]:
learning_rate = 0.01
batch_size = 128
n_epochs = 25

In [4]:
mnist = input_data.read_data_sets('./data/mnist', one_hot=True) 

Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz


In [5]:
n_train,ip_dim = mnist.train.images.shape
n_test,_ = mnist.test.images.shape
n_val,_ = mnist.validation.images.shape
_,op_dim = mnist.train.labels.shape
hidden_dim = 800
print n_train,n_test,n_val,ip_dim,op_dim,hidden_dim

55000 10000 5000 784 10 800


In [6]:
X = tf.placeholder(tf.float32,shape=[batch_size,ip_dim],name="X")
Y = tf.placeholder(tf.float32,shape=[batch_size,op_dim],name="Y")

In [7]:
w1 = tf.Variable(tf.random_normal(shape=[ip_dim,hidden_dim],stddev=0.01),name="w1")
b1 = tf.Variable(tf.zeros([1,hidden_dim]),name="b1")
w2 = tf.Variable(tf.random_normal(shape=[hidden_dim,op_dim],stddev=0.01),name="w2")
b2 = tf.Variable(tf.zeros([1,op_dim]),name="b2")

In [8]:
h1 = tf.add(tf.matmul(X,w1),b1,name="h1")
a1 = tf.sigmoid(h1,name="a1")
h2 = tf.add(tf.matmul(a1,w2),b2,name="h2")
a2 = tf.sigmoid(h2,name="a2")

In [9]:
entropy = tf.nn.softmax_cross_entropy_with_logits(logits=a2,labels=Y,name="entropy")
loss = tf.reduce_mean(entropy,name="loss")

In [10]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)
init = tf.global_variables_initializer()

In [11]:
with tf.Session() as sess:
    #writer = tf.summary.FileWriter('./graphs',sess.graph)
    sess.run(init)
    start_time = time.time()
    n_batches = int(n_train / batch_size)
    for i in range(n_epochs):
        total_loss = 0.0
        for _ in range(n_batches):
            X_batch,Y_batch = mnist.train.next_batch(batch_size)
            _,l = sess.run([optimizer,loss],feed_dict={X:X_batch,Y:Y_batch})
            total_loss += l
        print "Epoch: ",i,", average loss: ",(total_loss / n_batches)
        
    print "Total training time: ",(time.time() - start_time)," seconds"
    
    n_batches = int(n_test / batch_size)
    total_correct_pred = 0
    for _ in range(n_batches):
        X_batch,Y_batch = mnist.test.next_batch(batch_size)
        _,l,s = sess.run([optimizer,loss,a2],feed_dict={X:X_batch,Y:Y_batch})
        prob = tf.nn.softmax(s,name="prob")
        correct_pred = tf.equal(tf.argmax(prob,1),tf.argmax(Y_batch,1),name="correct_pred")
        num_correct_pred = tf.reduce_sum(tf.cast(correct_pred,tf.float32),name="num_correct_pred")
        total_correct_pred += sess.run(num_correct_pred)
    print "Accuracy: ",(total_correct_pred / n_test)
#writer.close()

Epoch:  0 , average loss:  2.30020505954
Epoch:  1 , average loss:  2.29704526588
Epoch:  2 , average loss:  2.29396765882
Epoch:  3 , average loss:  2.29039098889
Epoch:  4 , average loss:  2.28620241516
Epoch:  5 , average loss:  2.28110081499
Epoch:  6 , average loss:  2.27485662518
Epoch:  7 , average loss:  2.26711575301
Epoch:  8 , average loss:  2.25746712985
Epoch:  9 , average loss:  2.24564247976
Epoch:  10 , average loss:  2.23113106403
Epoch:  11 , average loss:  2.2137881359
Epoch:  12 , average loss:  2.19373578625
Epoch:  13 , average loss:  2.17100291541
Epoch:  14 , average loss:  2.14622756818
Epoch:  15 , average loss:  2.11989925236
Epoch:  16 , average loss:  2.09305336926
Epoch:  17 , average loss:  2.06619708855
Epoch:  18 , average loss:  2.04009951328
Epoch:  19 , average loss:  2.01537333585
Epoch:  20 , average loss:  1.99219869595
Epoch:  21 , average loss:  1.97101764496
Epoch:  22 , average loss:  1.95153756464
Epoch:  23 , average loss:  1.93389676695
Epo